In [4]:
!pip install mtcnn

In [7]:
!pip install keras-facenet

  Using cached keras-facenet-0.3.2.tar.gz (10 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10404 sha256=4c10f23e572dae71ce1b80d5e8fcc8c50119c05f575d751335c5847a5c0bfc5e
  Stored in directory: c:\users\rizal\appdata\local\pip\cache\wheels\73\5d\41\90b5d28ca667cfc4748ae859fa4f0b85b936d73207a073ded5
Successfully built keras-facenet


In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam
from mtcnn import MTCNN
import keras_facenet
from keras.applications import ResNet50

In [13]:
# Path ke folder dataset di Google Drive milik topik
dataset_path = 'Datasets/data_begin'

In [14]:
# Fungsi untuk mendeteksi dan mengekstrak wajah dari gambar
def extract_faces(image_path, required_size=(160, 160)):
    image = Image.open(image_path)
    pixels = np.array(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)

    if len(results) == 0:
        return None

    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = img_to_array(image)
    return face_array


In [15]:
# Path untuk menyimpan wajah yang diekstraksi
extracted_faces_path = 'Datasets/extracted_faces'

if not os.path.exists(extracted_faces_path):
    os.makedirs(extracted_faces_path)


In [16]:
# Ekstraksi wajah dari semua gambar di dataset
for category in os.listdir(dataset_path):
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for image_file in os.listdir(category_path):
            image_path = os.path.join(category_path, image_file)
            face_array = extract_faces(image_path)
            if face_array is not None:
                save_path = os.path.join(extracted_faces_path, category)
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                face_image = Image.fromarray(np.uint8(face_array))
                face_image.save(os.path.join(save_path, image_file))

print("Proses ekstraksi wajah selesai.")




1/1 [==============================] - 1s 663ms/step
Proses ekstraksi wajah selesai.


In [17]:
# Pengaturan Image Data Generator untuk augmentasi
augment_datagen = ImageDataGenerator(
    rescale=1.0/255,
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    channel_shift_range=30.0,
    fill_mode='nearest'
)

# Fungsi untuk melakukan augmentasi
def augment_images(input_folder, num_augmented_images):
    image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]
    num_original_images = len(image_files)
    num_augmentations_per_image = (num_augmented_images - num_original_images) // num_original_images

    for image_file in image_files:
        img_path = os.path.join(input_folder, image_file)
        img = load_img(img_path)  # Memuat gambar
        x = img_to_array(img)     # Konversi gambar ke array
        x = np.expand_dims(x, axis=0)  # Tambahkan dimensi batch

        # Buat iterator augmentasi
        aug_iter = augment_datagen.flow(x, batch_size=1)

        # Generate dan simpan gambar yang diaugmentasi
        for i in range(num_augmentations_per_image):
            batch = next(aug_iter)
            augmented_img = batch[0]
            augmented_img = np.uint8(augmented_img * 255)  # Skala kembali ke range [0, 255]
            augmented_img_path = os.path.join(input_folder, f"aug_{os.path.splitext(image_file)[0]}_{i}.jpg")
            augmented_img_pil = Image.fromarray(augmented_img)
            augmented_img_pil.save(augmented_img_path)

    # Jika masih kurang, augmentasi sisa gambar
    remaining_augments = num_augmented_images - num_original_images * (num_augmentations_per_image + 1)
    for i in range(remaining_augments):
        img_file = image_files[i % num_original_images]
        img_path = os.path.join(input_folder, img_file)
        img = load_img(img_path)  # Memuat gambar
        x = img_to_array(img)     # Konversi gambar ke array
        x = np.expand_dims(x, axis=0)  # Tambahkan dimensi batch

        aug_iter = augment_datagen.flow(x, batch_size=1)
        batch = next(aug_iter)
        augmented_img = batch[0]
        augmented_img = np.uint8(augmented_img * 255)  # Skala kembali ke range [0, 255]
        augmented_img_path = os.path.join(input_folder, f"aug_{os.path.splitext(img_file)[0]}_extra_{i}.jpg")
        augmented_img_pil = Image.fromarray(augmented_img)
        augmented_img_pil.save(augmented_img_path)

# Augmentasi gambar untuk setiap folder dalam folders_to_augment
folders_to_augment = ['Tendik', 'Dosen']
for folder in folders_to_augment:
    input_folder = os.path.join(extracted_faces_path, folder)
    augment_images(input_folder, num_augmented_images=200)

print("Proses augmentasi selesai.")

Proses augmentasi selesai.


In [18]:
!pip install tensorflow
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [19]:


# Pengaturan Image Data Generator dengan augmentasi dan ekstraksi wajah
img_height, img_width = 160, 160
batch_size = 64

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    brightness_range=[0.8, 1.2]
)

train_generator = train_datagen.flow_from_directory(
    extracted_faces_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    extracted_faces_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 464 images belonging to 3 classes.
Found 115 images belonging to 3 classes.


In [20]:
# Import the necessary module
from sklearn.utils import class_weight

# Extract class labels for all instances in the training dataset
classes = np.array(train_generator.classes)

# Calculate class weights to handle imbalances in the training data
# 'balanced' mode automatically adjusts weights inversely proportional to class frequencies
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',  # Strategy to balance classes
    classes=np.unique(classes),  # Unique class labels
    y=classes  # Class labels for each instance in the training dataset
)

# Create a dictionary mapping class indices to their calculated weights
class_weights_dict = dict(enumerate(class_weights))

# Output the class weights dictionary
print("Class Weights Dictionary:", class_weights_dict)


Class Weights Dictionary: {0: 0.9789029535864979, 1: 1.0175438596491229, 2: 1.0043290043290043}


In [21]:
# Load model FaceNet
facenet_model = keras_facenet.FaceNet()

# Definisikan model FaceNet
base_model = facenet_model.model
base_model.trainable = False

In [22]:
# Import the necessary module
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, Dense, Dropout

# Tambahkan layer custom
from tensorflow.keras.layers import Reshape, GlobalAveragePooling2D, Dense, Dropout
x = base_model.output
x = Reshape((1, 1, -1))(x)  # Reshape to (batch, 1, 1, features)
x = GlobalAveragePooling2D()(x)  # Now compatible with GlobalAveragePooling2D
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation='softmax')(x)  # Asumsikan kita punya 3 kelas

In [23]:
# Definisikan model
model = Model(inputs=base_model.input, outputs=predictions)

In [24]:
# Unfreeze beberapa layer terakhir dari base model FaceNet
for layer in base_model.layers[-60:]:
    layer.trainable = True

In [25]:
# Kompilasi ulang model
model.compile(optimizer=Adam(learning_rate=0.00001),  # Mengurangi learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
# Import the necessary module
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Tambahkan callbacks untuk early stopping dan reduce learning rate on plateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [27]:
# Latih model dengan epochs lebih banyak dan class weights
history = model.fit(
    train_generator,
    epochs=30,  # Tambahkan epochs
    validation_data=validation_generator,
    class_weight=class_weights_dict,  # Tambahkan class weights
    callbacks=[early_stopping, reduce_lr]  # Tambahkan early stopping dan reduce learning rate on plateau
)

In [ ]:
# Visualisasi hasil pelatihan
plt.figure(figsize=(12, 4))

# Plot akurasi pelatihan dan validasi
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss pelatihan dan validasi
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Evaluasi model pada data pelatihan
Y_pred_train = model.predict(train_generator)
y_pred_train = np.argmax(Y_pred_train, axis=1)
y_true_train = train_generator.classes

# Evaluasi model pada data validasi
Y_pred_val = model.predict(validation_generator)
y_pred_val = np.argmax(Y_pred_val, axis=1)
y_true_val = validation_generator.classes

class_labels = list(validation_generator.class_indices.keys())

# Menghitung dan menampilkan confusion matrix untuk data pelatihan
cm_train = confusion_matrix(y_true_train, y_pred_train)
print("Confusion Matrix - Training Data")
print(cm_train)

# Menampilkan confusion matrix sebagai heatmap untuk data pelatihan
plt.figure(figsize=(10, 8))
sns.heatmap(cm_train, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Training Data')
plt.show()

# Menghitung dan menampilkan confusion matrix untuk data validasi
cm_val = confusion_matrix(y_true_val, y_pred_val)
print("Confusion Matrix - Validation Data")
print(cm_val)

# Menampilkan confusion matrix sebagai heatmap untuk data validasi
plt.figure(figsize=(10, 8))
sns.heatmap(cm_val, annot=True, fmt='d', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix - Validation Data')
plt.show()

# Menghitung dan menampilkan classification report untuk data pelatihan
print("Classification Report - Training Data")
report_train = classification_report(y_true_train, y_pred_train, target_names=class_labels)
print(report_train)

# Menghitung dan menampilkan classification report untuk data validasi
print("Classification Report - Validation Data")
report_val = classification_report(y_true_val, y_pred_val, target_names=class_labels)
print(report_val)